# Description

#### Project  <span style="color:blue">NLP for divorce court decision documents</span> 

#### Content  <span style="color:blue">Word2Vec Clustering</span> 

#### Notebook Language  <span style="color:blue">python</span> 

 

In [1]:
import sys
sys.path.insert(0, "/usr/local/lib/python3.5/dist-packages")
import re #for string operations
import pandas as pd #for dataframes
from gensim.models import Word2Vec #Word2Vec
import numpy as np
from sklearn.cluster import DBSCAN

In [2]:
from kodex_nlp import stem_vocab, diff, read_files #user functions

# <span style="color:blue">1</span>  Read .pdf Documents from Path

## bozma = read_files("bozma/")
bozma_onama = read_files("bozma-onama/")
all_docs = bozma + bozma_onama
joined_docs = ' '.join(all_docs)
myFullList = joined_docs.split()
myList = list(set(joined_docs.split()))

In [4]:
print("Number of documents =",len(all_docs))
print("Total number of words in all documents =", len(myFullList))
print("Total number of unique words in all documents =", len(myList))

Number of documents = 26
Total number of words in all documents = 3393
Total number of unique words in all documents = 800


# <span style="color:blue">2</span>  Words in Word2Vec Vocablary

In [5]:
model = Word2Vec.load('tr/tr.bin')
vocab = model.wv.vocab

### Words in W2V vocablary:

In [6]:
print(str(100*np.round(len(list(filter(lambda x: x in vocab, myFullList)))/len(myFullList),2)) + 
      "% of words are in Vocablary")

31.0% of words are in Vocablary


### Words in W2V vocablary after applying stemming to those who are not in vocablary:

In [7]:
Fwords, FwordsC = stem_vocab(myFullList, vocab)
print(str(100*np.round(len(Fwords)/len(myFullList),2)) + 
      "% of words are in Vocablary after apllying stemming")

63.0% of words are in Vocablary after apllying stemming


# <span style="color:blue">3</span>  Clustering with Word2Vec Similarity Matrix

In [8]:
words = list(set(Fwords))
n = len(words)
d_M = np.zeros((n, n))

for i in range(n-1):
    for j in range(i+1,n):
        d_M[i, j] = model.wv.similarity(words[i], words[j])   
        
d_M = d_M + d_M.T + np.diag(np.ones(n))

In [9]:
db = DBSCAN(eps=1.96, min_samples=5, ).fit(d_M)
labels = db.labels_
unique_labels = set(labels)
n_clusters_ = len(unique_labels) - (1 if -1 in labels else 0)

print("Number of Clusters:", n_clusters_)

Number of Clusters: 12


In [10]:
for curr_label in unique_labels:
      
    if curr_label == -1:
        print("number of noise = " + str(np.sum(labels == curr_label)))
        
    else:
        print(str(np.sum(labels == curr_label)) + " of them labeled as " + str(curr_label))
    print("===============================")

13 of them labeled as 0
14 of them labeled as 1
12 of them labeled as 2
6 of them labeled as 3
24 of them labeled as 4
8 of them labeled as 5
6 of them labeled as 6
7 of them labeled as 7
5 of them labeled as 8
7 of them labeled as 9
5 of them labeled as 10
5 of them labeled as 11
number of noise = 218


In [11]:
def get_words(words, label):
    return np.array(words)[labels == label]

### Sample Clusters returned from DBSCAN

#### some clusters worked as stemmers:

In [22]:
print(get_words(words, 3))
print("===========================================================")
print(get_words(words, 5))
print("===========================================================")
print(get_words(words, 6))
print("===========================================================")
print(get_words(words, 9))

['anlaşılmış' 'belirtilmiştir' 'düşünüldü' 'belirtilmiş' 'neden'
 'anlaşılmıştır']
['vermemesi' 'verilmesine' 'alınmadan' 'verilmiş' 'verilip' 'alınmasına'
 'verilerek' 'düzenlenmesine']
['kapsamına' 'yasaya' 'kanuna' 'durumlarına' 'esasına' 'ilkesine']
['edilmek' 'edildiğine' 'edilemeyecek' 'edilecek' 'edilmeden' 'edileceğini'
 'edebileceği']


#### biggest cluster:

In [23]:
print(get_words(words, 4)) # some clusters worked as stemmers

['ücret' 'hükmü' 'temyiz' 'tazminat' 'yargılama' 'boşanma' 'kanununun'
 'bedel' 'davaya' 'mahkeme' 'duruşma' 'kararların' 'usul' 'uyuşmazlık'
 'dilekçe' 'uygulanmasını' 'borç' 'mevzuat' 'davalar' 'kanun' 'davaları'
 'kararlarını' 'kurul' 'dava']


#### noise words:

In [24]:
print(get_words(words, -1))

['usulü' 'düzenlemesi' 'rahatsızlığı' 'inceleme' 'i' 'açıldığı' 'toplanıp'
 'derecesini' 'yürürlükte' 'ayrılık' 'gerekirken' 'yön' 'bahis' 'küfür'
 'değer' 'koşullarının' 'dosyası' 'devamında' 'haiz' 'fiil' 'korunma'
 'yönlere' 'taktirde' 'dük' 'süresinde' 're' 'dönmesini' 'geçilerek'
 'dinlenen' 'gönderilmiş' 'imkân' 'sine' 'be' 'sıfat' 'bozulmuş' 'hastalık'
 'birleştirilmiştir' 'yöndeki' 'alışkanlığı' 'savunmanın' 'görgü'
 'bölümlerin' 'boşluk' 'makam' 'birliğinin' 'taraf' 'semt' 'hüküm' 'çoktur'
 'oluşturulmuş' 'düzenlemeleri' 'koyma' 'birliğin' 'ayrılmak' 'verilmek'
 'zina' 'oluşmuştur' 'gününün' 'avukatlık' 'geçirilerek' 'bulunmamaktadır'
 'karar' 'eldeki' 'kağıt' 'sağlayıcı' 'gerçekleştiği' 'şartlarını'
 'anlatım' 'mümkünse' 'niteliğindedir' 'azdır' 'yapılmadığı' 'ya' 'belge'
 'doğurur' 'ilkeler' 'nazaran' 'şartlarının' 'yoruma' 'gerek' 'usulüne'
 'aşaması' 'kuşkusuz' 'görülmemiş' 'çözüm' 'vekil' 'alacak' 'müşterek'
 'bozma' 'bölümlerinin' 'yasemin' 'birleştirilerek' 'oluşturmak'